In [1]:
#pip install selenium

In [1]:
import json
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup

from requests import get
import requests

from selenium import webdriver
import time

from random import randint
from time import sleep

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [3]:
df.head(60)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
5,A27UF1MSF3DB2,0000031887,"C-Lo ""Cynthia""","[0, 0]",I received this today and I'm not a fan of it ...,4.0,It's ok,1396224000,"03 31, 2014"
6,A16GFPNVF4Y816,0000031887,design maven,"[0, 0]",Bought this as a backup to the regular ballet ...,5.0,Great for dress-up and for ballet practice,1399075200,"05 3, 2014"
7,A2M2APVYIB2U6K,0000031887,Jamie P.,"[0, 0]",Great tutu for a great price. It isn't a &#34;...,5.0,Great value,1356220800,"12 23, 2012"
8,A1NJ71X3YPQNQ9,0000031887,JBerger,"[0, 0]","My daughter liked this, and it with her costum...",4.0,Good,1384041600,"11 10, 2013"
9,A3EERSWHAI6SO,0000031887,"Jeffrey Hollingshead ""Jillian hollingshead""","[7, 8]",For what I paid for two tutus is unbeatable an...,5.0,WOW !! ..is all I have to say!,1349568000,"10 7, 2012"


In [4]:
#Check size of dataset 
df.shape

(278677, 9)

In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
df[df['asin'] == 'B00971NLBS']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
224886,A1J4GHI1VRLCX9,B00971NLBS,Alya,"[1, 1]",I will be purchasing more of this brand of shi...,5.0,Classic,1374624000,"07 24, 2013"
224887,A21UXFFQGQS4YJ,B00971NLBS,"Anne Harris ""Lady H""","[0, 0]",I found it to be a little roomy but it will be...,5.0,nice dress shirt,1393545600,"02 28, 2014"
224888,A22Z5LSEIPN8DU,B00971NLBS,Ashleyy,"[0, 0]","I ordered this shirt in a light blue, size two...",2.0,"Wrong color, wrong size.",1372291200,"06 27, 2013"
224889,A3SP43WB89B2UE,B00971NLBS,"cecilafayette ""ceci""","[0, 0]","This shirt fits well, looks nice, and is a gre...",4.0,Great Little Basic Shirt,1388102400,"12 27, 2013"
224890,A1Q3MDPIS017JE,B00971NLBS,Kay Z,"[0, 0]","The material is a little thin, but that was ex...",4.0,"Good fit, little thin",1400284800,"05 17, 2014"
224891,A1BCJXOINBPB38,B00971NLBS,"lascrucesgirl ""Growing boy's momma""","[3, 4]",I did not use the tie for my son. It was dress...,5.0,Nicely made and looks sharp.,1360627200,"02 12, 2013"
224892,A1AFORE8L2LGGW,B00971NLBS,mandamommyof3,"[5, 5]",This is adorable! I would buy more if the tie...,4.0,Tie is too short,1356652800,"12 28, 2012"
224893,A22E4B018G6NX7,B00971NLBS,RDeering,"[0, 0]",Nice little shirt I couldn't find elsewhere th...,5.0,can't complain,1388966400,"01 6, 2014"


In [7]:
#Check how many unique items are in the data 
len(df['asin'].unique())

23033

In [8]:
#Created a non-repeating list of product ID #s with the intent of looping through them in the url to scrape. 
asin_set = set(df['asin'])
asin_set = list(asin_set)

In [10]:
asin_set.sort()

In [11]:
asin_set

['0000031887',
 '0123456479',
 '1608299953',
 '1617160377',
 'B00001W0KA',
 'B00001WRHJ',
 'B00004SR8W',
 'B00004SR8Z',
 'B00004SR9P',
 'B00004U1J2',
 'B000051SEN',
 'B000051SEP',
 'B00005JHKE',
 'B00005JSBK',
 'B00005KJXN',
 'B00005TQI7',
 'B0000643Q8',
 'B000067R84',
 'B00006I551',
 'B00006XXGO',
 'B0000722HK',
 'B000072UMJ',
 'B000074RL3',
 'B000078CYM',
 'B00007AS82',
 'B00007FFL9',
 'B00007GD8X',
 'B00007GD9I',
 'B00007GD9W',
 'B00007GDAL',
 'B00007GDD3',
 'B00007GDG5',
 'B00007IVVR',
 'B000086211',
 'B0000862FI',
 'B0000864CZ',
 'B0000865II',
 'B0000866JI',
 'B000086778',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B0000868O9',
 'B000086910',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B0000891IO',
 'B0000891K0',
 'B00008AALU',
 'B00008ECKG',
 'B00008I8YM',
 'B00008ID0B',
 'B00008ID0L',
 'B00008ID1O',
 'B00008ID39',
 'B00008IEUW',
 'B00008IOHR',
 'B00008IPCG',
 'B00008KH9C',
 'B00008KI3U',
 'B00008LUS5',
 'B00009ESZI',
 'B00009OXE8',
 'B00009QB5V',
 'B00009QMR3',
 'B00009R8

In [12]:
#Check how many unique users are in the data. 
len(df['reviewerID'].unique())

39387

EDA

In [13]:
#Check columns types 
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [14]:
#Convert universal time to date
df['clean_time'] = pd.to_datetime(df['unixReviewTime'], unit='s')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,clean_time
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011",2011-02-12
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013",2013-01-19
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013",2013-01-04
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014",2014-04-27
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014",2014-03-15


In [15]:
#Drop unuseful date and time columns 
df.drop(columns=['reviewTime', 'unixReviewTime'], inplace=True)

In [16]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'clean_time'],
      dtype='object')

In [17]:
df['reviewText'].head()

0    This is a great tutu and at a really great pri...
1    I bought this for my 4 yr old daughter for dan...
2    What can I say... my daughters have it in oran...
3    We bought several tutus at once, and they are ...
4    Thank you Halo Heaven great product for Little...
Name: reviewText, dtype: object

Scrape additional data using Selenium

In [51]:
driver = webdriver.Chrome(executable_path="../chromedriver/macos/chromedriver") 

In [18]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [52]:
t1 = time.time()
print(t1)
list_of_dicts = []
for i in redo_list[10000:]:
    print(f'https://www.amazon.com/gp/product/{i}')
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()

        product_info['description'] = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-none'}).text.strip()
        
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).text.strip()
        
        product_info['size'] = soup.find('div', {'id': 'variation_size_name'})
        
        product_info['details'] = soup.find_all('td', {'class': 'a-span7 a-size-base'})
        
        if soup.find('div', {'id': 'variation_color_name'}):
            product_info['color'] = soup.find('div', {'id': 'variation_color_name'})
            list_of_dicts.append(product_info)
        
        elif soup.find_all('span', {'class': 'selection'}):
            product_info['color'] = soup.find_all('span', {'class': 'selection'})
            list_of_dicts.append(product_info)
            
        elif soup.find_all('div', {'class': 'a-row a-spacing-micro'}):
            product_info['color'] = soup.find_all('div', {'class': 'a-row a-spacing-micro'})
            list_of_dicts.append(product_info)
        else:
            product_info['color'] = 'unavailable'
            list_of_dicts.append(product_info)
    
    except:
        product_info['asin'] = i
        
        product_info['name'] = 'unknown product name'

        product_info['description'] = 'unknown details'
        
        product_info['category'] = 'unknown category'
        
        product_info['color'] = 'unknown color' 
        list_of_dicts.append(product_info)
        print(len(list_of_dicts))
    sleep(randint(1,5))
    

print(time.time() - t1)

1565674568.505593
https://www.amazon.com/gp/product/B00B8SDHSU
https://www.amazon.com/gp/product/B00B8TSCUW
https://www.amazon.com/gp/product/B00B8VFIEI
https://www.amazon.com/gp/product/B00B8X5X5A
https://www.amazon.com/gp/product/B00B8XGCSW
5
https://www.amazon.com/gp/product/B00B8XYOAA
6
https://www.amazon.com/gp/product/B00B8Y1ZTM
7
https://www.amazon.com/gp/product/B00B8YCLNG
8
https://www.amazon.com/gp/product/B00B93W5R8
9
https://www.amazon.com/gp/product/B00B9FNUMU
10
https://www.amazon.com/gp/product/B00B9JJL42
11
https://www.amazon.com/gp/product/B00B9RZ9EK
12
https://www.amazon.com/gp/product/B00B9VWVXS
13
https://www.amazon.com/gp/product/B00BAHU6V0
14
https://www.amazon.com/gp/product/B00BAIRM16
15
https://www.amazon.com/gp/product/B00BANCUGI
16
https://www.amazon.com/gp/product/B00BAPZEQ4
17
https://www.amazon.com/gp/product/B00BAWUPWU
18
https://www.amazon.com/gp/product/B00BAWZMG4
19
https://www.amazon.com/gp/product/B00BB7RCU2
20
https://www.amazon.com/gp/product/B00BB

170
https://www.amazon.com/gp/product/B00BUT640U
171
https://www.amazon.com/gp/product/B00BV7ALAK
172
https://www.amazon.com/gp/product/B00BVD00DW
173
https://www.amazon.com/gp/product/B00BVJ7UDE
174
https://www.amazon.com/gp/product/B00BW0UJI0
175
https://www.amazon.com/gp/product/B00BW899MO
176
https://www.amazon.com/gp/product/B00BW9YUUY
177
https://www.amazon.com/gp/product/B00BWEO6L2
178
https://www.amazon.com/gp/product/B00BXJ1R54
179
https://www.amazon.com/gp/product/B00BXJ1UJW
180
https://www.amazon.com/gp/product/B00BXOBPMY
181
https://www.amazon.com/gp/product/B00BXP0C90
182
https://www.amazon.com/gp/product/B00BXP3YBS
183
https://www.amazon.com/gp/product/B00BXPMAK4
184
https://www.amazon.com/gp/product/B00BXT7ZMI
185
https://www.amazon.com/gp/product/B00BY6HLDI
186
https://www.amazon.com/gp/product/B00BY6I5KG
187
https://www.amazon.com/gp/product/B00BY6ID58
188
https://www.amazon.com/gp/product/B00BY7UNR8
189
https://www.amazon.com/gp/product/B00BYE6PNC
190
https://www.amaz

https://www.amazon.com/gp/product/B00CBG7RY8
345
https://www.amazon.com/gp/product/B00CBLQGWW
346
https://www.amazon.com/gp/product/B00CBPFA48
347
https://www.amazon.com/gp/product/B00CBPY5AS
348
https://www.amazon.com/gp/product/B00CBSVQX4
https://www.amazon.com/gp/product/B00CBTD3GQ
https://www.amazon.com/gp/product/B00CBWM04Y
https://www.amazon.com/gp/product/B00CBWOLZA
352
https://www.amazon.com/gp/product/B00CC6Q0W2
353
https://www.amazon.com/gp/product/B00CCA5P4C
https://www.amazon.com/gp/product/B00CCM4IHA
355
https://www.amazon.com/gp/product/B00CCM4IIY
https://www.amazon.com/gp/product/B00CCVM670
357
https://www.amazon.com/gp/product/B00CCWXPHE
358
https://www.amazon.com/gp/product/B00CD4A1UA
359
https://www.amazon.com/gp/product/B00CD7BYB2
360
https://www.amazon.com/gp/product/B00CD9SZA8
https://www.amazon.com/gp/product/B00CDAQ82Y
https://www.amazon.com/gp/product/B00CDAYQUK
363
https://www.amazon.com/gp/product/B00CDB8JTI
364
https://www.amazon.com/gp/product/B00CDBMSBI
htt

519
https://www.amazon.com/gp/product/B00CM27DDQ
https://www.amazon.com/gp/product/B00CM2APTA
https://www.amazon.com/gp/product/B00CM2OBDG
https://www.amazon.com/gp/product/B00CM5YHTQ
https://www.amazon.com/gp/product/B00CMDWUIS
https://www.amazon.com/gp/product/B00CMEZ7NW
525
https://www.amazon.com/gp/product/B00CMEZB1A
https://www.amazon.com/gp/product/B00CMLQXM4
https://www.amazon.com/gp/product/B00CMNQ8MM
528
https://www.amazon.com/gp/product/B00CMO8CCU
https://www.amazon.com/gp/product/B00CMQ915A
https://www.amazon.com/gp/product/B00CMQ9F38
https://www.amazon.com/gp/product/B00CMQ9KQK
https://www.amazon.com/gp/product/B00CMR8UJM
https://www.amazon.com/gp/product/B00CMTF8XQ
534
https://www.amazon.com/gp/product/B00CMV68QA
535
https://www.amazon.com/gp/product/B00CMXNNMA
https://www.amazon.com/gp/product/B00CMXZEOA
https://www.amazon.com/gp/product/B00CMXZFUS
538
https://www.amazon.com/gp/product/B00CN38IAG
539
https://www.amazon.com/gp/product/B00CN47GXA
540
https://www.amazon.com/

691
https://www.amazon.com/gp/product/B00CWK61A4
692
https://www.amazon.com/gp/product/B00CWK6FM8
693
https://www.amazon.com/gp/product/B00CWLY2W2
694
https://www.amazon.com/gp/product/B00CWR3FZG
695
https://www.amazon.com/gp/product/B00CX9AG7S
696
https://www.amazon.com/gp/product/B00CXKQ12G
697
https://www.amazon.com/gp/product/B00CXLBA2G
698
https://www.amazon.com/gp/product/B00CXMN0GY
699
https://www.amazon.com/gp/product/B00CXPEJ9I
700
https://www.amazon.com/gp/product/B00CXSWQ7C
701
https://www.amazon.com/gp/product/B00CXVECQW
702
https://www.amazon.com/gp/product/B00CXWF2YC
703
https://www.amazon.com/gp/product/B00CXXID2O
704
https://www.amazon.com/gp/product/B00CXYMVFI
705
https://www.amazon.com/gp/product/B00CY28MMU
706
https://www.amazon.com/gp/product/B00CY8ON68
707
https://www.amazon.com/gp/product/B00CYAVKIA
708
https://www.amazon.com/gp/product/B00CYNAUE2
709
https://www.amazon.com/gp/product/B00CYU6SEG
710
https://www.amazon.com/gp/product/B00CZ20CF4
711
https://www.amaz

...

1193
https://www.amazon.com/gp/product/B00EL3LHBS
1194
https://www.amazon.com/gp/product/B00EL8HWBM
1195
https://www.amazon.com/gp/product/B00EL8LT6G
https://www.amazon.com/gp/product/B00ELPXKES
1197
https://www.amazon.com/gp/product/B00ELQPNQA
1198
https://www.amazon.com/gp/product/B00ELQVZGC
1199
https://www.amazon.com/gp/product/B00ELZPJ24
1200
https://www.amazon.com/gp/product/B00ELZPQJK
1201
https://www.amazon.com/gp/product/B00ELZQ1QC
1202
https://www.amazon.com/gp/product/B00ELZQMGQ
1203
https://www.amazon.com/gp/product/B00EM1USF0
1204
https://www.amazon.com/gp/product/B00EM4ORFY
1205
https://www.amazon.com/gp/product/B00EM4P3IY
1206
https://www.amazon.com/gp/product/B00EMB67VY
1207
https://www.amazon.com/gp/product/B00EMBUJC2
1208
https://www.amazon.com/gp/product/B00EMG3PNC
https://www.amazon.com/gp/product/B00EMG3PO6
1210
https://www.amazon.com/gp/product/B00EMG3VZE
1211
https://www.amazon.com/gp/product/B00EN9IQ0A
1212
https://www.amazon.com/gp/product/B00ENCGGZY
1213
https

1359
https://www.amazon.com/gp/product/B00F0M7JUW
1360
https://www.amazon.com/gp/product/B00F0X5FA2
1361
https://www.amazon.com/gp/product/B00F13N828
1362
https://www.amazon.com/gp/product/B00F13NABC
1363
https://www.amazon.com/gp/product/B00F13RJ6O
1364
https://www.amazon.com/gp/product/B00F1H3TZK
1365
https://www.amazon.com/gp/product/B00F20FU7G
1366
https://www.amazon.com/gp/product/B00F2B4U44
1367
https://www.amazon.com/gp/product/B00F2C8RFQ
1368
https://www.amazon.com/gp/product/B00F2K1RSW
https://www.amazon.com/gp/product/B00F2L57G4
1370
https://www.amazon.com/gp/product/B00F2OFWHK
1371
https://www.amazon.com/gp/product/B00F2V77JE
https://www.amazon.com/gp/product/B00F2V77K8
1373
https://www.amazon.com/gp/product/B00F2VI2SO
1374
https://www.amazon.com/gp/product/B00F2ZX4XS
1375
https://www.amazon.com/gp/product/B00F35FJ3U
1376
https://www.amazon.com/gp/product/B00F3AWAJ6
1377
https://www.amazon.com/gp/product/B00F3AWAJQ
1378
https://www.amazon.com/gp/product/B00F3D230Y
1379
https

1525
https://www.amazon.com/gp/product/B00FLXMST2
1526
https://www.amazon.com/gp/product/B00FM3J1XM
1527
https://www.amazon.com/gp/product/B00FMEENCA
1528
https://www.amazon.com/gp/product/B00FMF1WDW
1529
https://www.amazon.com/gp/product/B00FMJIR5E
https://www.amazon.com/gp/product/B00FMMG5G4
https://www.amazon.com/gp/product/B00FMZ8WLW
1532
https://www.amazon.com/gp/product/B00FMZERVG
1533
https://www.amazon.com/gp/product/B00FN5ASIG
1534
https://www.amazon.com/gp/product/B00FN6UDUS
https://www.amazon.com/gp/product/B00FNFVSGC
1536
https://www.amazon.com/gp/product/B00FNHL8UQ
1537
https://www.amazon.com/gp/product/B00FNN8NBW
1538
https://www.amazon.com/gp/product/B00FNN8NC6
https://www.amazon.com/gp/product/B00FNN8OA2
1540
https://www.amazon.com/gp/product/B00FNN8PMO
1541
https://www.amazon.com/gp/product/B00FNPPQS8
1542
https://www.amazon.com/gp/product/B00FNSIJWU
1543
https://www.amazon.com/gp/product/B00FNZHA32
1544
https://www.amazon.com/gp/product/B00FO3AQ9I
https://www.amazon.c

1691
https://www.amazon.com/gp/product/B00GJW29DO
1692
https://www.amazon.com/gp/product/B00GJWE3JW
1693
https://www.amazon.com/gp/product/B00GJXITVE
1694
https://www.amazon.com/gp/product/B00GKC1KC4
1695
https://www.amazon.com/gp/product/B00GKC9P7Q
1696
https://www.amazon.com/gp/product/B00GKC9P9O
1697
https://www.amazon.com/gp/product/B00GKDAYKC
1698
https://www.amazon.com/gp/product/B00GKDB8KM
1699
https://www.amazon.com/gp/product/B00GKDBOH4
1700
https://www.amazon.com/gp/product/B00GL6GMVS
1701
https://www.amazon.com/gp/product/B00GLJCJPI
1702
https://www.amazon.com/gp/product/B00GLXHM6K
1703
https://www.amazon.com/gp/product/B00GM0MD0C
1704
https://www.amazon.com/gp/product/B00GM0ZGMY
1705
https://www.amazon.com/gp/product/B00GM4129Q
1706
https://www.amazon.com/gp/product/B00GM4ADIC
1707
https://www.amazon.com/gp/product/B00GM5MFHI
1708
https://www.amazon.com/gp/product/B00GM67AH2
1709
https://www.amazon.com/gp/product/B00GM7UROE
1710
https://www.amazon.com/gp/product/B00GMC9OLG


1856
https://www.amazon.com/gp/product/B00HTS8N0K
1857
https://www.amazon.com/gp/product/B00HTZHXSQ
1858
https://www.amazon.com/gp/product/B00HUOESOS
1859
https://www.amazon.com/gp/product/B00HUOQUM6
1860
https://www.amazon.com/gp/product/B00HUZW7PY
1861
https://www.amazon.com/gp/product/B00HV6O5DY
1862
https://www.amazon.com/gp/product/B00HVEY87E
1863
https://www.amazon.com/gp/product/B00HVWSQG0
https://www.amazon.com/gp/product/B00HVXXEOI
1865
https://www.amazon.com/gp/product/B00HWF7KL8
1866
https://www.amazon.com/gp/product/B00HWF7KPY
1867
https://www.amazon.com/gp/product/B00HWF7KR2
1868
https://www.amazon.com/gp/product/B00HWRSQQE
1869
https://www.amazon.com/gp/product/B00HX0CSL4
1870
https://www.amazon.com/gp/product/B00HY2T5TO
1871
https://www.amazon.com/gp/product/B00HYT82E6
1872
https://www.amazon.com/gp/product/B00HZRJLGA
1873
https://www.amazon.com/gp/product/B00I07HQMU
1874
https://www.amazon.com/gp/product/B00I0QK3R6
1875
https://www.amazon.com/gp/product/B00I0XXR5E
1876


In [53]:
#Check the length of the scrape 
len(list_of_dicts)

2009

In [54]:
#Check the know number of "unknown" rows to make sure it's low 
pd.DataFrame(list_of_dicts)['category'].value_counts()

unknown category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              1688
Clothing, Shoes & Jewelry\n            \n\n\n             ›\n         \n\n\n                Women\n            \n\n\n             ›\n         \n\n\n                Clothing\n            \n\n\n             ›\n         \n\n\n                Dresses\n            \n\n\n             ›\n         \n\n\n                 Casual                                                                                                                                                                                11
Cl

In [14]:
# Save first 8000 scrapes to a DataFrame and then to a csv
# product8000 = pd.DataFrame(list_of_dicts)

# product8000.to_csv('product8000.csv')

In [22]:
# Saved next 534 scrapes to a DataFrame and then to a csv
# product8534 = pd.DataFrame(list_of_dicts)

# product8534.to_csv('data/product8534')

In [29]:
# Saved next 6000 scrapes to a DataFrame and then to a csv
# product14000 = pd.DataFrame(list_of_dicts)

# product14000.to_csv('product14000.csv')

In [35]:
# Saved another 6000 scrapes to a DataFrame and then to a csv
# product20000 = pd.DataFrame(list_of_dicts)

# product20000.to_csv('product20000.csv')

In [41]:
# Saved next 1180 scrapes to a DataFrame and then to a csv
# product21180 = pd.DataFrame(list_of_dicts)

# product21180.to_csv('product21180.csv')

In [46]:
# Saved last section of scrapes to a DataFrame and then to a csv

# product23033 = pd.DataFrame(list_of_dicts)

# product23033.to_csv('product23033.csv')

Many of the above scrapes were blocked by an anti-robot page so i'll need to go through and check for unavailable rows and make a new list of asins to rescrape.

In [18]:
# Function to remove html
def remove_html(text):
    try:
        soup = BeautifulSoup(text, 'lxml')
        html_free = soup.get_text(strip=True)
        return html_free
    except:
        return "missing"

In [19]:
# Function to clean the DataFrame
def EDA(df):
    # Remove html and strip off brackets and 'Color:' from color column
    df['color'] = df['color'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '').str.replace('Color:', '')
    
    # Replace escape characters and white space in Category column 
    df['category'] = df['category'].str.replace('\n', '').str.replace('  ', '')
    
    # Split Categroy column on the '›' symbol, up to 6 times and return them in a new df where each split is a new column 
    category = df['category'].str.split("›", n=6, expand=True)
    
    # Rename each category split column and add it onto the original df
    df['department'] = category[0]
    df['demographic'] = category[1]
    df['division'] = category[2]
    df['category'] = category[3]
    df['subcategory'] = category[4]
    df['type'] = category[5]
    df['detail_type'] = category[6]
    
    # Remove special characters from the description column 
    df['description'] = df['description'].str.replace('\t', '').str.replace('\n', '')
    
    # Remove html and strip off brackets from details column
    df['details'] = df['details'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '')
   
    #Remove html and cut off 'Size:' from the size column 
    df['size'] = df['size'].apply(lambda x: remove_html(x)).str.replace('Size:', '')
    
    return df.head()

In [20]:
product8000 = pd.read_csv('data/product8000.csv')

In [21]:
product20000 = pd.read_csv('data/product20000.csv')

In [22]:
product21180 = pd.read_csv('data/product21180.csv')

In [23]:
product23033 = pd.read_csv('data/product23033.csv')

In [24]:
#A list of DataFrames to be concatonated
dataframes = [product8000, product20000, product21180, product23033]

# Concat all 4 DataFrames into one, along the row axis.
total_product = pd.concat(dataframes, axis=0)

In [25]:
total_product.shape

(17033, 8)

In [26]:
EDA(total_product)

,Unnamed: 0,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,0,0000031887,Dance,unavailable,This fits your . Make sure this fitsby ent...,,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,missing,Sports & Outdoors,Sports & Fitness,Other Sports,Clothing,Girls,Skirts
1,1,0123456479,Jewelry Boxes & Organizers,unavailable,This fits your . Make sure this fitsby ent...,,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,missing,"Clothing, Shoes & Jewelry","Shoe, Jewelry & Watch Accessories",Jewelry Accessories,Jewelry Boxes,None,None
2,2,1608299953,None,unavailable,Access for up to 5 family members Download act...,,Learn French: Rosetta Stone French - Level 1,missing,Software,Education & Reference,Languages,None,None,None
3,3,1617160377,None,unavailable,Access for up to 5 family members Download act...,,Learn Italian: Rosetta Stone Italian - Level 1,missing,Software,Education & Reference,Languages,None,None,None
4,4,B00001W0KA,None,Buzz Lightyear,Lead Free Child (4-6 & 7-8) Includes: Bodysuit...,,Buzz Lightyear Boy's Deluxe Toy Story Costume,SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...,Toys & Games,Dress Up & Pretend Play,Costumes,None,None,None


Create a dataframe for only rows that came back unknown.

In [27]:
redo_df = total_product[total_product['department'] == 'unknown category']

In [28]:
redo_df.shape

(12009, 14)

In [29]:
redo_df.isnull().sum()

Unnamed: 0         0
asin               0
category       12009
color              0
description        0
details            0
name               0
size               0
department         0
demographic    12009
division       12009
subcategory    12009
type           12009
detail_type    12009
dtype: int64

In [30]:
redo_list = list(redo_df['asin'])
redo_list

['B00005TQI7',
 'B0000722HK',
 'B00007GD9I',
 'B0000864CZ',
 'B0000866JI',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B00008ID0B',
 'B00008ID1O',
 'B00008ID39',
 'B00008IOHR',
 'B00008KI3U',
 'B00008LUS5',
 'B00009QB5V',
 'B0000AT3NT',
 'B0000B35D8',
 'B0000TIIRS',
 'B0000X5EJO',
 'B0000X5FAM',
 'B0000YEC8C',
 'B0000YECC8',
 'B0000ZCH4W',
 'B000147UNU',
 'B00014861K',
 'B00018HRU2',
 'B0001IA06U',
 'B0001VJ2V6',
 'B0001VWSFS',
 'B0001WD5RC',
 'B0001XH7X4',
 'B0001YWBG6',
 'B000200JQI',
 'B00020ZUC6',
 'B00023K3YS',
 'B00029BKAI',
 'B0002DG7ZM',
 'B0002EJ4XS',
 'B0002FHFZQ',
 'B0002FHGBO',
 'B0002FHIM6',
 'B0002FHIMQ',
 'B0002FHIVM',
 'B0002FHIWQ',
 'B0002JA0WC',
 'B0002L01YC',
 'B0002LKLPQ',
 'B0002NZ0LO',
 'B0002TVB70',
 'B0002U71R8',
 'B0002VG2XG',
 'B0002XROVS',
 'B0002ZOQZ8',
 'B00031XFL2',
 'B00061RG1O',
 'B00063J8MW',
 'B00063W3GA',
 'B00064UU1Y',
 'B000662EL6',
 'B00066G516',
 'B000673JT6',
 'B00068VGOU',
 'B0006GK8V4',
 'B0006IXC

In [31]:
len(redo_list)

12009

In [36]:
# redo2000 = pd.DataFrame(list_of_dicts)


In [34]:
redo2000.isnull().sum()

NameError: name 'redo2000' is not defined

In [35]:
EDA(redo2000)

NameError: name 'redo2000' is not defined

In [50]:
more_redos = (redo2000[redo2000['department'] == 'unknown category'])
redo_list2 = list(more_redos['asin'])
len(redo_list2)

1402

In [33]:
# Save all rows that are not unknown to a csv
#(redo2000[redo2000['department'] != 'unknown category']).to_csv('data/redo1.csv')

In [43]:
#redo6000 = pd.DataFrame(list_of_dicts)
redo6000.head()

,asin,category,color,description,details,name,size
0,B0018NHASI,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% cotton. Machine wash and dry. Imported.\n...,[],Jockey Men's Underwear Classic Boxer Brief - 6...,"<div class=""a-section a-spacing-base variation..."
1,B0018NHD9Y,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",This fits your .\n \n\n\n Make sure this fi...,[],Body Glove Men's Riptide Water Shoe,"<div class=""a-section a-spacing-small"" id=""var..."
2,B0018ODO9G,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Cotton\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n...,[],Levi's Men's Silver Tab Baggy Jean,"<div class=""a-section a-spacing-base variation..."
3,B0018OFRK0,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Cotton\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \n...,[],Levi's Men's Big and Tall 501 Original Fit Jean,"<div class=""a-section a-spacing-base variation..."
4,B0018OFTA8,"Clothing, Shoes & Jewelry\n \n\n\n ...","<div class=""a-section a-spacing-small"" id=""var...",100% Leather\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n \...,[],Levi's Men's Leather Belt With Plaque Buckle,"<div class=""a-section a-spacing-base variation..."


In [45]:
redo6000.isnull().sum()

asin              0
category          0
color             0
description       0
details        2811
name              0
size           3212
dtype: int64

In [46]:
EDA(redo6000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B0018NHASI,Underwear,missing,100% cotton. Machine wash and dry. Imported. U...,missing,Jockey Men's Underwear Classic Boxer Brief - 6...,missing,"Clothing, Shoes & Jewelry",Men,Clothing,Boxer Briefs,None,None
1,B0018NHD9Y,Shoes,missing,This fits your . Make sure this fitsby ent...,missing,Body Glove Men's Riptide Water Shoe,missing,"Clothing, Shoes & Jewelry",Men,"Surf, Skate & Street",Athletic,Water Shoes,None
2,B0018ODO9G,Jeans,missing,100% Cotton Imported Machine Wash Whether it's...,missing,Levi's Men's Silver Tab Baggy Jean,missing,"Clothing, Shoes & Jewelry",Men,Clothing,None,None,None
3,B0018OFRK0,Jeans,missing,100% Cotton Imported Button Fly closure Wash A...,missing,Levi's Men's Big and Tall 501 Original Fit Jean,missing,"Clothing, Shoes & Jewelry",Men,Clothing,None,None,None
4,B0018OFTA8,Novelty,missing,100% Leather PERFECT BELT FOR YOUR NEXT BARBEC...,missing,Levi's Men's Leather Belt With Plaque Buckle,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Men,Accessories,Belt Buckles


In [52]:
redo_list2.extend(list((redo6000[redo6000['department'] == 'unknown category'])['asin']))
len(redo_list2)

4213

In [53]:
# Save the rescraped rows that are not unknown to a csv
# (redo6000[redo6000['department'] != 'unknown category']).to_csv('data/redo2.csv')

In [21]:
#redo6455 = pd.DataFrame(list_of_dicts)

In [22]:
# Save functiont
# EDA(redo6455)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B007JBJPOA,"Lingerie, Sleep & Lounge",missing,nylon shapewear-briefs closure Light control f...,missing,Barely There Second Skinnies Smoothers Boxer,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Lingerie,Shapewear,Thigh Slimmers
1,B007JEHD5A,Wigs,unavailable,Products: 1 long Straight wig (100cm/39') + Wi...,missing,Hunnt® Long Cosplay Party Red Straight Wig 100cm,missing,"Clothing, Shoes & Jewelry",Costumes & Accessories,Women,None,None,None
2,B007JEI2N2,Pumps,missing,100% Leather Synthetic sole Heel measures appr...,missing,Naturalizer Women's Joni Pump,missing,"Clothing, Shoes & Jewelry",Women,Shoes,None,None,None
3,B007JF0UEK,Pants,missing,"94% Cotton, 6% Spandex Imported Machine Wash S...",missing,Lee Women's Perfect Fit Nancy Barely Bootcut Pant,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Casual,None,None
4,B007JF0UFO,Pants,unavailable,"Black, Carbon Rinse, Indigo Rinse, Raven, Whit...",missing,Lee Women's Midrise Vivian Trouser,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Casual,None,None


In [50]:
# Save rows that are not unknown to a csv

# (redo6455[redo6455['department'] != 'unknown category']).to_csv('data/redo3.csv')

In [37]:
# Save rescrape to a DataFrame 
# redo8528 = pd.DataFrame(list_of_dicts)

# Save the cleaned data to a csv
# redo8528.to_csv('redo_list1.csv')

# Read in the cleaned data
redo8528 = pd.read_csv('redo_list1.csv', index_col=[0])
redo8528.shape

(2073, 13)

In [22]:
# Use funtion to clean rescrape 
# EDA(redo8528)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B007WA2TCS,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
1,B007WA2U7W,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
2,B007WA2VB2,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
3,B007WA2ZTA,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B007WA3558,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [41]:
redo8528['category'].value_counts(dropna=False)

NaN                              1146
 Athletic                          98
 Sandals                           70
 Lingerie, Sleep & Lounge          55
Wrist Watches                      39
 Boots                             35
Pumps                              30
Flats                              30
 Earrings                          29
 Tops, Tees & Blouses              26
 Novelty                           25
 Clothing                          20
Fashion Sneakers                   19
 Scarves & Wraps                   18
 Jackets & Coats                   18
Jeans                              18
 Bracelets                         17
 Dresses                           16
 Running                           15
 Necklaces                         14
 Shirts                            13
 Active                            12
Loafers & Slip-Ons                 12
 Pants                             10
 Underwear                         10
 Sweaters                          10
 Skirts     

In [42]:
# check how many product names were collected  
(redo8528[redo8528['name'] != 'unknown product name']).shape

(950, 13)

In [23]:
# Save only rows that have department values to a new csv to be joined on to the clean data in a different notebook.
# (redo8528[redo8528['department'] != 'unknown category']).to_csv('data/redo4.csv')

In [43]:
# Check if number of categories collected matches names
(redo8528[redo8528['department'] != 'unknown category']).shape

(950, 13)

In [48]:
# Saved rescrape 
#redo10000 = pd.DataFrame(list_of_dicts)

In [38]:
# Used function to clean rescraped data
# EDA(redo10000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B009L4OGDS,Earrings,unavailable,6mm Crystal Ball Stud Earrings Plain Post and ...,missing,Sterling Silver Pink Crystal Ball Stud Earrings,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Stud,None,None
1,B009L5081Q,Novelty,unavailable,Size: One size only (small) Total Length: Abou...,missing,Ninimour- Fashion Trendy Women's Stretchy Legg...,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Women,Leggings,None
2,B009L53QXI,Boots,missing,Synthetic Made in USA or Imported Synthetic so...,missing,Dark Tan Faux Leather Bold Red Back Zipper Buc...,missing,"Clothing, Shoes & Jewelry",Women,Shoes,Mid-Calf,None,None
3,B009L5YU7O,Shoulder Bags,missing,Leathette Imported fabric lining Snap closure ...,missing,MG Collection Yelena Top Handle Soft Hobo Shou...,missing,"Clothing, Shoes & Jewelry",Women,Handbags & Wallets,None,None,None
4,B009LB8CX6,Active,missing,Made in USA or Imported Merino wool blend Fibe...,missing,Kirkland Signature Outdoor Trail Sock Merino W...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Athletic Socks,None,None


In [44]:
# Save known column names to a csv
# (redo10000[redo10000['name'] != 'unknown product name']).to_csv('data/redo5.csv')

In [43]:
(redo10000[redo10000['name'] == 'unknown product name'])

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
8,B009LEFPAQ,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
10,B009LEFR4A,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
11,B009LEFS26,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
12,B009LEG2EO,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
13,B009LEG3XO,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
16,B009LM7BWS,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
17,B009LNATJY,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
22,B009LZTK10,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
24,B009M8UF38,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
28,B009MAW0A2,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [45]:
# Save clean rescraped data to a csv
#redo10000.to_csv('redo_list2.csv')

redo10000 = pd.read_csv('redo_list2.csv', index_col=[0])
redo10000.head()

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B009L4OGDS,Earrings,unavailable,6mm Crystal Ball Stud Earrings Plain Post and ...,missing,Sterling Silver Pink Crystal Ball Stud Earrings,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Stud,NaN,NaN
1,B009L5081Q,Novelty,unavailable,Size: One size only (small) Total Length: Abou...,missing,Ninimour- Fashion Trendy Women's Stretchy Legg...,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Women,Leggings,NaN
2,B009L53QXI,Boots,missing,Synthetic Made in USA or Imported Synthetic so...,missing,Dark Tan Faux Leather Bold Red Back Zipper Buc...,missing,"Clothing, Shoes & Jewelry",Women,Shoes,Mid-Calf,NaN,NaN
3,B009L5YU7O,Shoulder Bags,missing,Leathette Imported fabric lining Snap closure ...,missing,MG Collection Yelena Top Handle Soft Hobo Shou...,missing,"Clothing, Shoes & Jewelry",Women,Handbags & Wallets,NaN,NaN,NaN
4,B009LB8CX6,Active,missing,Made in USA or Imported Merino wool blend Fibe...,missing,Kirkland Signature Outdoor Trail Sock Merino W...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Athletic Socks,NaN,NaN


In [46]:
redo_list2 = (list((redo8528[redo8528['department'] == 'unknown category'])['asin'])) + (list((redo10000[redo10000['department'] == 'unknown category'])['asin']))

len(redo_list2)


2099

In [47]:
redo_list2

['B007WA2TCS',
 'B007WA2U7W',
 'B007WA2VB2',
 'B007WA2ZTA',
 'B007WA3558',
 'B007WA38IM',
 'B007WA38OG',
 'B007WA397M',
 'B007WA3B5C',
 'B007WA3CUQ',
 'B007WA3GMK',
 'B007WA3HLK',
 'B007WA3JE0',
 'B007WA3PD0',
 'B007WA48E0',
 'B007WA49DA',
 'B007WA4CQ4',
 'B007WACT6Y',
 'B007WACX3I',
 'B007WACY5K',
 'B007WAD2QU',
 'B007WADBUC',
 'B007WADJ7M',
 'B007WADN4G',
 'B007WADRBK',
 'B007WADUOY',
 'B007WAEB10',
 'B007WAEBPQ',
 'B007WAEU5C',
 'B007WAEV9C',
 'B007WAF02E',
 'B007WAF8TO',
 'B007WAF9LQ',
 'B007WAFEWU',
 'B007WAFEYI',
 'B007WAFIEE',
 'B007WAFIGM',
 'B007WAFN4E',
 'B007WAFZTW',
 'B007WASL66',
 'B007WAT3I6',
 'B007WAT69W',
 'B007WAT8IG',
 'B007WATABQ',
 'B007WATBLU',
 'B007WATI18',
 'B007WAU1VY',
 'B007WAU2OK',
 'B007WB4OIY',
 'B007WN8MEE',
 'B007WO1PJW',
 'B007WPH9BY',
 'B007WPHD92',
 'B007WPHJRS',
 'B007WPHRVG',
 'B007WPHUYU',
 'B007WQY5W4',
 'B007WZJYTY',
 'B007X43KX0',
 'B007X44HTQ',
 'B007X44WV4',
 'B007X49DUY',
 'B007X734FG',
 'B007X73VAO',
 'B007XBE0BE',
 'B007XD4NCI',
 'B007XD4T

In [55]:
redo12009 = pd.DataFrame(list_of_dicts)

In [56]:
EDA(redo12009)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00B8SDHSU,Loafers & Slip-Ons,missing,100% Leather Imported Rubber sole Shaft measur...,missing,Clarks Men's Portland 2 Easy Slip-On,missing,"Clothing, Shoes & Jewelry",Men,Shoes,None,None,None
1,B00B8TSCUW,Hats & Caps,missing,Women's Light Beret Crochet Beanie Head Circum...,missing,Pop Fashionwear Women's Light Beret Knitted St...,missing,"Clothing, Shoes & Jewelry",Women,Accessories,Berets,None,None
2,B00B8VFIEI,Men,missing,cotton-blend Premium cotton for a lasting soft...,missing,Nike Premium Cotton Crew Cut Socks (6 Pack) La...,missing,Sports & Outdoors,Sports & Fitness,Clothing,Socks,None,None
3,B00B8X5X5A,Rings,missing,Width: 7 mm Lightweight and beautiful Packagin...,missing,Chuvora 925 Sterling Silver Woven Celtic Knot ...,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Bands,None,None
4,B00B8XGCSW,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [57]:
redo12009['name'].value_counts()

unknown product name                                                                                                                                                             1688
Kano Bak Women Quartz Fashion Weave Wrap around Leather Bracelet Wrist Watch                                                                                                        1
Gerber Baby-Girls Newborn 2 Pack Sleep N Play                                                                                                                                       1
BareTraps Women's Reeta Gladiator Sandal                                                                                                                                            1
WrightSock Women's Coolmesh II Lo 4 Pack                                                                                                                                            1
Scarfand's Feminine Lace Infinity Scarf with Teardrop Fringes                             

In [60]:
(redo12009[redo12009['name'] != 'unknown product name']).to_csv('data/redo6.csv')

In [61]:
redo12009.to_csv('redo_list3.csv')

In [64]:
redo12009 = pd.read_csv('redo_list3.csv', index_col=[0])
redo12009.head()

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00B8SDHSU,Loafers & Slip-Ons,missing,100% Leather Imported Rubber sole Shaft measur...,missing,Clarks Men's Portland 2 Easy Slip-On,missing,"Clothing, Shoes & Jewelry",Men,Shoes,NaN,NaN,NaN
1,B00B8TSCUW,Hats & Caps,missing,Women's Light Beret Crochet Beanie Head Circum...,missing,Pop Fashionwear Women's Light Beret Knitted St...,missing,"Clothing, Shoes & Jewelry",Women,Accessories,Berets,NaN,NaN
2,B00B8VFIEI,Men,missing,cotton-blend Premium cotton for a lasting soft...,missing,Nike Premium Cotton Crew Cut Socks (6 Pack) La...,missing,Sports & Outdoors,Sports & Fitness,Clothing,Socks,NaN,NaN
3,B00B8X5X5A,Rings,missing,Width: 7 mm Lightweight and beautiful Packagin...,missing,Chuvora 925 Sterling Silver Woven Celtic Knot ...,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Bands,NaN,NaN
4,B00B8XGCSW,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN


In [70]:
redo_list2.extend(list((redo12009[redo12009['name'] == 'unknown product name'])['asin']))


In [71]:
len(redo_list2)

3787